# titanic_datatset.py

In [1]:
import os, torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader, random_split
import wandb
import argparse
from datetime import datetime


/home/cv-05/anaconda3/envs/hy/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class TitanicDataset(Dataset): #Train 데이터
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X) #float 타입의 텐서 생성
        self.y = torch.FloatTensor(y) #Long 타입의 텐서 생성
    
    def __len__(self): 
        return len(self.X) #텐서의 길이 반환
    def __getitem__(self, idx):
        feature = self.X[idx] #idx를 이용하여 input값인 feature를 x에 레이블 값인 target을 y에 선언
        target = self.y[idx]
        return {'input': feature, 'target': target}
    def __str__(self):
        str = "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(len(self.X), self.X.shape, self.y.shape) #Data의 크기, feature의 shape, target의 shape를 str에 초기화
        return str

In [3]:
class TitanicTestDataset(Dataset): #Test 데이터
    def __init__(self, X):
        self.X = torch.FloatTensor(X) 
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        feature = self.X[idx]
        return {'input': feature}
    def __str__(self):
        str = "Data Size: {0}, Input Shape: {1}".format(len(self.X), self.X.shape)
        return str

In [4]:
def get_preprocessed_dataset_1(all_df): 
    # Pclass별 Fare 평균값을 사용하여 Fare 결측치 메우기
    Fare_mean = all_df[["Pclass", "Fare"]].groupby("Pclass").mean().reset_index() #Pclass열을 기준으로 Fare열의 평균값을 계산하여 인덱스를 재설정
    Fare_mean.columns = ["Pclass", "Fare_mean"] #Fare_mean의 열 이름을 Pclass와 Fare_mean으로 변경
    all_df = pd.merge(all_df, Fare_mean, on="Pclass", how="left")  #all_df데이터 프레임과 Fare_mean 열을 Pclass 열을 기준으로 병합
    all_df.loc[(all_df["Fare"].isnull()), "Fare"] = all_df["Fare_mean"]  #Fare열의 결측치가 있는 행을 선택하고 그 행의 Fare값을 Fare_mean의 값을 대입
    
    return all_df

In [5]:
def get_preprocessed_dataset_2(all_df):
    # name을 세 개의 컬럼으로 분리하여 다시 all_df에 합침
    name_df = all_df["Name"].str.split("[,.]", n=2, expand=True) #Name열의 데이터를 ,.를 기준으로 최대 2번 분할하고 새로운 데이터로 저장
    name_df.columns = ["family_name", "honorific", "name"] #분할한 데이터의 열 이름 지정
    name_df["family_name"] = name_df["family_name"].str.strip() #문자열의 앞뒤 공백 제거
    name_df["honorific"] = name_df["honorific"].str.strip()
    name_df["name"] = name_df["name"].str.strip()
    all_df = pd.concat([all_df, name_df], axis=1) #all_df와 name_df를 수평으로 concat

    return all_df

In [6]:
def get_preprocessed_dataset_3(all_df):
    # honorific별 Age 평균값을 사용하여 Age 결측치 메우기
    honorific_age_mean = all_df[["honorific", "Age"]].groupby("honorific").median().round().reset_index() #name의 존칭인 honorific과 Age를 그룹지어 중앙값 계산후 반올림하여 honorific_age_mean에 젖아
    honorific_age_mean.columns = ["honorific", "honorific_age_mean", ]
    all_df = pd.merge(all_df, honorific_age_mean, on="honorific", how="left")
    all_df.loc[(all_df["Age"].isnull()), "Age"] = all_df["honorific_age_mean"] #Age의 결측치를 honorific_age_mean으로 채움 
    all_df = all_df.drop(["honorific_age_mean"], axis=1) #all_df에 병합된 honorific_age_mean열을 제거

    return all_df

In [7]:
def get_preprocessed_dataset_4(all_df):
    # 가족수(family_num) 컬럼 새롭게 추가
    all_df["family_num"] = all_df["Parch"] + all_df["SibSp"] #부모/자녀수와 형제자매/배우자 수를 더한 새로운 열 family_num 추가

    # 혼자탑승(alone) 컬럼 새롭게 추가
    all_df.loc[all_df["family_num"] == 0, "alone"] = 1 #alone이라는 열을 생성하고 family_name이 0인 경우 1값을 저장
    all_df["alone"].fillna(0, inplace=True) # 결츨칙를 0으로 처리

    # 학습에 불필요한 컬럼 제거
    all_df = all_df.drop(["PassengerId", "Name", "family_name", "name", "Ticket", "Cabin"], axis=1) 

    return all_df

In [8]:
def get_preprocessed_dataset_5(all_df):
    # honorific 값 개수 줄이기
    all_df.loc[
    ~(
            (all_df["honorific"] == "Mr") |
            (all_df["honorific"] == "Miss") |
            (all_df["honorific"] == "Mrs") |
            (all_df["honorific"] == "Master")
    ),
    "honorific"
    ] = "other" #주어진 존칭이 아닌 경우 other honorific 값 저장
    all_df["Embarked"].fillna("missing", inplace=True) #탑승지의 결측치들은 missing으로 처리

    return all_df

In [9]:
def get_preprocessed_dataset_6(all_df):
  # 카테고리 변수를 LabelEncoder를 사용하여 수치값으로 변경하기
  category_features = all_df.columns[all_df.dtypes == "object"] #all_df의 열의 데이터 타입이 object인 열들을 저장
  from sklearn.preprocessing import LabelEncoder #문자열 데이터를 숫자로 인코딩하는 싸이킷 라이브러리
  for category_feature in category_features: 
    le = LabelEncoder()
    if all_df[category_feature].dtypes == "object":
      le = le.fit(all_df[category_feature]) 
      all_df[category_feature] = le.transform(all_df[category_feature]) #문자열 값을 정수 레이블로 변환

  return all_df

In [10]:
def get_preprocessed_dataset():
    CURRENT_FILE_PATH = os.path.dirname(os.path.abspath('./hw2.ipynb')) #hw2_titani.ipynb가 있는 파일의 경로를 저장
    train_data_path = os.path.join(CURRENT_FILE_PATH, "train.csv") #train.csv가 있는 경로 저장
    test_data_path = os.path.join(CURRENT_FILE_PATH, "test.csv") #test.csv가 있는 경로 저장
    
    train_df = pd.read_csv(train_data_path) #불러오기
    test_df = pd.read_csv(test_data_path)
    

    all_df = pd.concat([train_df, test_df], sort=False) #train_df와 test_df concat 열의 순서는 유지
    all_df = get_preprocessed_dataset_1(all_df) #데이터 전처리 함수1~6
    all_df = get_preprocessed_dataset_2(all_df)
    all_df = get_preprocessed_dataset_3(all_df)
    all_df = get_preprocessed_dataset_4(all_df)
    all_df = get_preprocessed_dataset_5(all_df)
    all_df = get_preprocessed_dataset_6(all_df)
    
    train_X = all_df[~all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True) #train_data의 레이블 값인 Survived가 결측치가 아닌 행을 선택해 input값인 feature를 x에 저장
    train_y = train_df["Survived"] #y는 레이블 값을 저장
    test_X = all_df[all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True) #test_data에서 레이블 값이 결측치인 행을 선택해 test데이터의 input값을 저장

    dataset = TitanicDataset(train_X.values, train_y.values)
    #print(dataset)
    train_dataset, validation_dataset = random_split(dataset, [0.8, 0.2]) #train_data를 8:2비율로 학습 데이터와 검증 데이터로 분리
    test_dataset = TitanicTestDataset(test_X.values)
    #print(test_dataset)

    return train_dataset, validation_dataset, test_dataset

In [11]:
from torch import nn, optim

class MyModel(nn.Module): #학습 모델 정의
    def __init__(self, n_input, n_output):
        super().__init__() 
        self.model = nn.Sequential( #레이어를 연속적으로 쌓음
            nn.Linear(n_input, 30),  #선형 레이어를 정의 뉴런의 개수는 30
            nn.LeakyReLU(0.1), #LeakyReLU 활성화 함수
            nn.Linear(30, 30), #총 2개의 뉴런 레이어층을 가짐
            nn.LeakyReLU(0.1),
            nn.Linear(30, n_output),
        )
    def forward(self, x): #순전파 과정 순서대로 레이어릁 통과
        x = self.model(x) #모델을 통과한 값 x를 저장
        return x 

In [12]:
def get_model_and_optimizer():
  my_model = MyModel(n_input=11, n_output=2)
  optimizer = optim.SGD(my_model.parameters(), lr=wandb.config.learning_rate) #SGD optim을 이용하여 weight를 업데이트함

  return my_model, optimizer

In [13]:
train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset() #학습,검증,테스트 데이터를 전처리후 불러오는 함수
train_data_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True) #batch_size가 16으로 한벅 반복 할 경우 16개씩 묶어서 학습을 하게됨
validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=16, shuffle=True)
test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

In [14]:
def training_loop(model, optimizer, train_data_loader, validation_data_loader, test_data_loader):
  n_epochs = wandb.config.epochs
  loss_fn = nn.MSELoss()  # Use a built-in loss function

  for epoch in range(1, n_epochs + 1):
    loss_train = 0.0
    num_trains = 0
    for train_batch in train_data_loader:
      output_train = model(train_batch['input'])
      output_train = torch.argmax(output_train, dim=1) 
      loss = loss_fn(output_train, train_batch['target'])
      loss.requires_grad = True
      loss_train += loss.item()
      num_trains += 1

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    loss_validation = 0.0
    num_validations = 0
    with torch.no_grad():
      for validation_batch in validation_data_loader:
        output_validation = model(validation_batch['input'])
        output_validation = torch.argmax(output_validation, dim=1) 
        loss = loss_fn(output_validation, validation_batch['target'])
        loss.requires_grad = True
        loss_validation += loss.item()
        num_validations += 1
    print(
      f"Epoch {epoch}, "
      f"Training loss {loss_train / num_trains:.4f}, "
      f"Validation loss {loss_validation / num_validations:.4f}"
    )

    wandb.log({
      "Epoch": epoch,
      "Training loss": loss_train / num_trains,
      "Validation loss": loss_validation / num_validations
    })

  CURRENT_FILE_PATH = os.path.dirname(os.path.abspath('./hw2.ipynb'))
  sub_path = os.path.join(CURRENT_FILE_PATH, "gender_submission.csv") #test.csv가 있는 경로 저장
  submission = pd.read_csv(sub_path) #불러오기
  with torch.no_grad():
    for test_batch in test_data_loader:
      output_test = model(test_batch['input'])
      output_test = torch.argmax(output_test, dim=1) 
      print("[TEST]")
      submission['Survived'] = output_test
      submission.to_csv('hw2.csv', index=False)
      print(output_test)

In [15]:
current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

config = {
    'epochs': 4,
    'batch_size': 32,
    'learning_rate': 0.01,
    'n_hidden_unit_list': [20, 20],
    }

wandb.init(
    mode="online" if True else "disabled",
     project="hw2",
     notes="DeepLearning",
    tags=["my_model", "california_housing"],
    name=current_time_str,
    config=config
  )
# print(args)
print(wandb.config)

my_model, optimizer = get_model_and_optimizer()

wandb.watch(my_model)

print("#" * 50, 1)
training_loop(
  model=my_model,
  optimizer=optimizer,
  train_data_loader=train_data_loader,
  validation_data_loader=validation_data_loader,
  test_data_loader=test_data_loader
  )
wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: wer9295. Use `wandb login --relogin` to force relogin


{'epochs': 4, 'batch_size': 32, 'learning_rate': 0.01, 'n_hidden_unit_list': [20, 20]}
################################################## 1
Epoch 1, Training loss 0.3582, Validation loss 0.3177
Epoch 2, Training loss 0.3582, Validation loss 0.3542
Epoch 3, Training loss 0.3549, Validation loss 0.3542
Epoch 4, Training loss 0.3549, Validation loss 0.3177
[TEST]
tensor([0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
        0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0,
        1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
        1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1,
        1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
        0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
        1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 

Epoch,▁▃▆█
Training loss,██▁▁
Validation loss,▁██▁
Epoch,4
Training loss,0.35494
Validation loss,0.31771


# 리더보드 제출 

![leaderboard.PNG](./leaderboard.PNG)

# 고찰

과제를 하면서 titanic 데이터셋을 분석하여 어떤 feature가 중요한지 그리고 결측치 처리 방법에 대해 알 수 있었다.  
  
하지만 f_my_model_training_with_wandb.py의 학습 모델과 학습하는 코드를 데이터에 맞게 수정하다 보니 오류가 많이 발생했다.  
  
일단 output의 dimension의 크기가 맞지 않아서 오류가 발생하여 output_test = torch.argmax(output_test, dim=1)를 추가했고 또 jupyter에서 wanb의 parser 문제가 발생하여 실행할 수 없었다. 
   
그래서 config의 값들을 각각 선언해주었다. 또한 test 메서드에서 submission.csv를 내보내는 코드를 작성했지만 my_model = MyModel(n_input=11, n_output=2) 이 코드가 model의 객체를 다시 불러와서 
학습이 진행되지 않고 랜덤한 값을 초기화만 했다.  
  
그래서 test 메서드를 없애고 학습하는 메서드에서 학스하여 나온 output값을 gender_submission.csv파일에 넣어주는 코드를 작성하였다.  
  
이렇게 오류를 해결하기 위해 코드를 자꾸 바꾸다 보니 교수님이 주신 코드에서 많이 바뀐 것 같다.   
  
그리고 epoch의 값을 여러가지 수행해 보았고 batch사이즈도 변경하여 실험을 해보았다. epoch이 6을 넘으면 overfitting이 일어나는 것을 확인 할 수 있엇다.  
  
그중 epoch은 4의 값이 loss가 가장 적었고 batch 사이즈는 32가 가장 loss가 적었다.  
  
오류를 줄이기 위해서 hidden_size를 더 늘리거나 layer의 층을 더 깊게 쌓는게 더 효율이 좋았다.
  
또한 더 적합한 learning_rate를 찾는다면 더욱 오류를 줄일 수 있을 것 같다.  
  
마지막으로 활성화 함수 ELU,Leaky ReLU, PReLUm ReLU등 여러가지 활성화 함수를 사용했는데 Leaky ReLU 활성화 함수가 가장 적은 loss를 보였다. 하지만 ReLU랑 크게 다르진 않았다.

# 과제를 하면서 느낀 점

예전에 daycon에서 주관하는 전력사용량 예측하는 대회에 참가해 본적이 있었다.  
  
model은 pytorch에서 불러오면 되었기 때문에 model 코드를 작성하는 부분에는 어려움이 없었다. 하지만 데이터를 분석하고 데이터를 전처리 하는 과정이 중요했었다. 
  
loss를 줄이기 위해 여러가지 활성화 함수도 사용하고 다양한 optim, hyperparmeter를 사용하여 결과를 냈었다.  
  
이번 과제도 마찬가지로 그런 부분에서 kaggle이란 대회에서 제공하는 데이터를 분석하고 전처리하는 과정 또 실험하는 코드가 있었는데 재미가 있었다.  
  
또한 wanb라는 사이트에 가입하였고 실제로 loss가 변하는 것을 그래프로 볼 수 있다면 앞으로 DeepLearning을 하면서 hyperparameter를 실험하는 과정이 많이 있을텐데 wanb 라이브러리를 자주 사용한다면 실험을 더욱 편리하게 할 수 있을 것 같다고 느꼈고 많은 것을 배울 수 있었던 과제인 것 같다.  